In [125]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [126]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)
df.drop('Unnamed: 0', axis=1, inplace=True)
df.drop(df.index[:1], inplace=True)

In [127]:
import re

# Fakten zum Datensatz
anzahl_sitzungen = len(all_files)
anzahl_reden = len(df.index)

# anzahl wörter (alle Wörter)
df_new = df[df['inhalt_neu'].notnull()]

def count_words(speech):
    words = speech.split()
    word_count = len(words)
    return word_count

dfc = df_new.copy()
dfc['word_count'] = dfc.apply(lambda row: count_words(row['inhalt_neu']), axis=1)

words_total = dfc['word_count'].sum()

# Anzahl des Wortes "Klimawandel"
def count_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimawandel", string)
    return len(result)

def count_any_climate_regex(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    resultKlima = re.findall(r"klima\w+", string)
    resultUmwelt = re.findall(r"umwelt\w+", string)#noch genauer suchen
    return len(resultKlima) + len(resultUmwelt)

def count_klimahysterie(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klimahysterie", string)
    return len(result)


dfc['Anzahl Klimawandel'] = dfc.apply(lambda row: count_climate(row['inhalt_neu']), axis=1)
dfc['Anzahl Klimahysterie'] = dfc.apply(lambda row: count_klimahysterie(row['inhalt_neu']), axis=1) 

climate_total = dfc['Anzahl Klimawandel'].sum() 
klimahysterie_total = dfc['Anzahl Klimahysterie'].sum()

def count_any_climate(speech):
    words = speech.split()
    string = ''.join(words)
    string = str(string)
    result = re.findall("klima", string) #noch genauer suchen
    return len(result)

#any_climate_total = dfc['Anzahl Klima'].sum()

In [4]:
n2_df = dfc.loc[dfc['Anzahl Klimahysterie'] != 0]

In [128]:
import numpy as np

# Filtern auf Datensätze zu Klima und Umwelt
dfc['Anzahl Klima/Umwelt'] = dfc.apply(lambda row: count_any_climate_regex(row['inhalt_neu']), axis=1) 

n3_df = dfc.loc[dfc['Anzahl Klima/Umwelt'] != 0]
n3_df = n3_df.replace(['BÜNDNIS\xa090/DIE GRÜNEN'],['BÜNDNIS 90/DIE GRÜNEN'])
n3_df = n3_df.replace(['BÜNDIS 90/DIE GRÜNEN'],['BÜNDNIS 90/DIE GRÜNEN'])
n3_df = n3_df.replace(['BÜND'],['BÜNDNIS 90/DIE GRÜNEN'])

#Filtern auf Parteien

n3_df = n3_df.loc[n3_df['fraktion'] == 'BÜNDNIS 90/DIE GRÜNEN']
print(len(n3_df))


# Fraktionen zählen
spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

def count_parties(party, my_list): 
    if party == "SPD":
        my_list[0] +=1
        return my_list
    elif party == "CDU/CSU":
        my_list[1] +=1
        return my_list
    elif party == "FDP":
        my_list[2] +=1
        return my_list
    elif party == "DIE LINKE":
        my_list[3] +=1
        return my_list
    elif party == "AfD":
        my_list[4] +=1
        return my_list
    elif bool(re.search("DIE GRÜNEN", str(party))) == True: 
        my_list[5] +=1
        return my_list
    elif party == 'fraktionslos' or party == 'Fraktionslos':
        my_list[6] +=1
        return my_list
    return


result = n3_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

912


In [129]:
n4_df = dfc.loc[dfc['Anzahl Klimawandel'] != 0]
len(n4_df.index)

spd_counter = cdu_counter = fdp_counter = linke_counter = afd_counter = grüne_counter = fraktionslos_counter = 0 
parties_list = [spd_counter, cdu_counter, fdp_counter, linke_counter, afd_counter, grüne_counter, fraktionslos_counter]

result2 = n4_df.apply(lambda row: count_parties(row['fraktion'], parties_list), axis=1)

In [130]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
"""

fig, ax = plt.subplots(figsize = (25,17))    
fig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, 
                  estimator = sum, ci = None, ax=ax)

x_dates = pd.to_datetime(dfc['datum'], dayfirst=True).sort_values().unique()
ax.set_xticklabels(labels=x_dates, rotation=45, ha='right')

"""

'\n\nfig, ax = plt.subplots(figsize = (25,17))    \nfig = sns.barplot(x = "datum", y = "Anzahl Klima", data = dfc, \n                  estimator = sum, ci = None, ax=ax)\n\nx_dates = pd.to_datetime(dfc[\'datum\'], dayfirst=True).sort_values().unique()\nax.set_xticklabels(labels=x_dates, rotation=45, ha=\'right\')\n\n'

In [131]:
import gensim
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Preprocessing Topic Modeling

stop_words = stopwords.words('german')
manual_stop_words = (['werd','herr','kolleg','mehr','woll','wer', 'gut', 'wichtig','uber','konn','sag','frag', 'sag',
                      'frau', 'herr''schon', 'wurd', 'gibt', 'thema', 'ganz', 'mal', 'konn', 'glaub', 'gesagt', 'mach',
                      'geht', 'stell', 'all', 'and', 'red', 'hatt', 'debatt','mocht', 'dank', 'word', 'lieb', 'letzt',
                      'find', 'darub', 'darauf', 'desweg','eigent', 'vielleicht', 'genau', 'gar','bundesregier','deutsch', 'europa', 'mensch', 'deutschland', 'jahr', 
                     'antrag', 'sag', 'land', 'polit', 'minist', 'milliard', 'wurd', 'dam', 'uber', 'hat', 'ganz', 'prozent', 'mehr','braucht', 'frau', 'neu'])
stop_words = stop_words + manual_stop_words
stemmer = SnowballStemmer('german')
def tokenize_and_stem(text):
    
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        token = stemmer.stem(token)
        if len(token) > 2 and token not in stop_words:
            filtered_tokens.append(token)
    return filtered_tokens
long_string = n3_df.inhalt_neu.values.tolist()


In [132]:
import gensim.corpora as corpora
data_words = []
for speech in long_string:
    data_words.append(list(tokenize_and_stem(speech)))


In [133]:
#LDA Topic Modeling

id2word = corpora.Dictionary(data_words)
texts = data_words

In [134]:
corpus = [id2word.doc2bow(text) for text in texts]

In [135]:
from pprint import pprint
from gensim import models
num_topics = 10
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
#word2vec = models.Word2Vec(corpus)

In [136]:
lda_model = gensim.models.LdaMulticore(corpus=corpus_tfidf,
                                       id2word=id2word,
                                       num_topics=num_topics)

In [120]:
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

[(0,
  '0.001*"haushalt" + 0.000*"brauch" + 0.000*"bahn" + 0.000*"endlich" + '
  '0.000*"union" + 0.000*"wied" + 0.000*"ausbau" + 0.000*"gemeinsam" + '
  '0.000*"antwort" + 0.000*"klimaschutz"'),
 (1,
  '0.000*"wirtschaft" + 0.000*"haushalt" + 0.000*"euro" + '
  '0.000*"schuldenbrems" + 0.000*"investition" + 0.000*"endlich" + '
  '0.000*"konkret" + 0.000*"unternehm" + 0.000*"erneuerbar" + 0.000*"ehrlich"'),
 (2,
  '0.001*"haushalt" + 0.001*"euro" + 0.000*"klimaschutz" + 0.000*"erneuerbar" '
  '+ 0.000*"endlich" + 0.000*"gesetz" + 0.000*"gesetzentwurf" + '
  '0.000*"investition" + 0.000*"union" + 0.000*"nachhalt"'),
 (3,
  '0.000*"haushalt" + 0.000*"kris" + 0.000*"schul" + 0.000*"regier" + '
  '0.000*"union" + 0.000*"massnahm" + 0.000*"geld" + 0.000*"afd" + '
  '0.000*"ziel" + 0.000*"geg"'),
 (4,
  '0.000*"digitalisier" + 0.000*"euro" + 0.000*"haushalt" + 0.000*"innovation" '
  '+ 0.000*"ministerin" + 0.000*"million" + 0.000*"nachhalt" + 0.000*"sozial" '
  '+ 0.000*"forschung" + 0.000*"

In [137]:
import pyLDAvis
import pickle
import pyLDAvis.gensim_models

# Visualierung der Topics

pyLDAvis.enable_notebook()

LDAvis_data_filepath =  os.path.join('./results/ldavis_prepared_'+str(num_topics))

In [138]:
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
   

/opt/tljh/user/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/tljh/user/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/tljh/user/lib/python3.9/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: dis

In [123]:
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.019666 -0.002247       1        1  30.708217
4      0.002873  0.012997       2        1  15.306178
3     -0.000629  0.006032       3        1  13.385264
0      0.001877  0.001849       4        1   8.934265
6     -0.000088  0.005070       5        1   8.490678
9      0.001724 -0.002147       6        1   6.465816
1      0.002660 -0.000734       7        1   5.789485
8      0.003849 -0.005275       8        1   4.518644
7      0.002319 -0.008755       9        1   3.323623
5      0.005081 -0.006789      10        1   3.077832, topic_info=             Term       Freq      Total Category  logprob  loglift
567    wirtschaft  63.000000  63.000000  Default  30.0000  30.0000
1664         wald  35.000000  35.000000  Default  29.0000  29.0000
6479         wolf  12.000000  12.000000  Default  28.0000  28.0000
447   beschaftigt  32.000000  32.000000  Default  27.0000  27.0000
406          euro  93.000000  93.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
214         dafur   1.749097  64.700217  Topic10  -8.3144  -0.1297
177    erneuerbar   1.778494  76.536946  Topic10  -8.2977  -0.2811
219           end   1.682972  50.365963  Topic10  -8.3529   0.0822
1031     haushalt   1.726612  98.387701  Topic10  -8.3273  -0.5618
14         brauch   1.673730  71.907025  Topic10  -8.3584  -0.2794

[762 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
10998      1  0.270271  -kilometer-abstandsregel
10998      2  0.135136  -kilometer-abstandsregel
10998      3  0.135136  -kilometer-abstandsregel
10998      4  0.135136  -kilometer-abstandsregel
10998      5  0.135136  -kilometer-abstandsregel
...      ...       ...                       ...
3156       3  0.099304                  zuzulass
3156       4  0.297911                  zuzulass
3156       5  0.099304                  zuzulass
3156       8  0.099304                  zuzulass
3156      10  0.099304                  zuzulass

[4148 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 1, 7, 10, 2, 9, 8, 6])

In [44]:

from lda2vec import preprocess, Corpus

2022-06-17 16:26:54.602800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-17 16:26:54.602863: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


ImportError: cannot import name 'preprocess' from 'lda2vec' (/opt/tljh/user/lib/python3.9/site-packages/lda2vec/__init__.py)